In [ ]:
import os, glob
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import linear_model
import statsmodels.api as sm

#Tests the model for accuracy
from sklearn.model_selection import train_test_split

#Scales outliers so that it doesnt affect the data too much
from sklearn.preprocessing import StandardScaler

#Model Creation
from sklearn.neighbors import KNeighborsClassifier

#The last three tools are for testing the model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score, mean_squared_error, f1_score
from sklearn.metrics import accuracy_score

#This will be used to fill in the missing data
from sklearn.impute import SimpleImputer


In [ ]:
#Removes the "\" from the player name
def Player_name_fix(ht):
    #Fixes Player name to remove "\"
    ht_ = ht.split("\\")
    KeptPortion = ht_[0]
    return KeptPortion

#Changes height from feet into inches
def feet_to_inches(ht):
    #Split the height between ft and inches
    ht_ = ht.split("-")
    #Convert to float
    ft_ = float(ht_[0])
    in_ = float(ht_[1])
    ft_to_in = (12*ft_) + in_
    return ft_to_in

#https://www.youtube.com/watch?v=dcQs8k9WGbY
#This function will combine the csv files of the "combine" datasets and create a new file with combined elements
def combine_datasets(folder_location, folder_name):
    master_combined = pd.DataFrame()
    #Folder location exactly where stored on computer
    for file in os.listdir(folder_location):
        if file.endswith('.csv'):
            #Loops through all files and appends to the dataframe created before
            master_combined = master_combined.append(pd.read_csv(".\\"+folder_name+"\\" + file))
        
    master_combined.to_csv('combined_everything.csv', index=False)

#https://www.youtube.com/watch?v=ooj84UP3r6M&t=404s   
#This function will scrape this website for draft information and place that information into workable dataframes   
def webscrapingDraft(start_year, end_year):
    site = "https://www.pro-football-reference.com/years/{}/draft.htm"
    url = site.format(start_year)
    
    #Creates an initial dataframe from start year so that with every loop, new data is appended at the end
    #Turns site info into list
    Scraped_Start = pd.read_html(url, header = [1])
    #Turns list into dataframe, remove other tables not needed
    Scraped_Start = Scraped_Start[0]
    
    #While loop to scrape data and append into a single CSV file
    while start_year <= end_year: 
        url = site.format(start_year+1)
        #Turns site info into list
        scraped_Next = pd.read_html(url, header = [1])
        #Turns list into dataframe, remove other tables not needed
        scraped_Next = scraped_Next[0]
        start_year+=1
        #Append to the end of the dataframe at the end of the loop
        Scraped_Start = Scraped_Start.append(scraped_Next, ignore_index=True)
    
    #Dataset had repeating headers so find all headers aside from the first and drop
    #Shows where age is repeated
    Scraped_Start[Scraped_Start.Age == 'Age']
    #Drop rows where header is reshown
    Scraped_Start = Scraped_Start.drop(Scraped_Start[Scraped_Start.Age == 'Age'].index)
    
    #Save data onto computer
    Scraped_Start.to_csv('draft'+str(start_year)+'.csv', index=False)
    
    #Creation of a segment for people who were drafted, drops all the rest
    df_scraped_segment = Scraped_Start[['Player','Pos',]]
    
    #Adding column for drafted
    df_scraped_segment["Drafted"] = "True"
    
    df_scraped = df_scraped_segment
    return df_scraped

#Merges two dataframes and removes/adjusts any parts that hamper the modeling
def merge_and_clean(scraped_website, combine_dataframe):
    #Cleaning the names from the combine dataset
    combine_dataframe["Player"] = combine_dataframe["Player"].apply(Player_name_fix)
    
    #Merging combine and scraped website
    mergedDataFrame = pd.merge(scraped_website, combine_dataframe, on='Player', how='right')
    
    #Filling in "False" for those not drafted that year
    mergedDataFrame[['Drafted']] = mergedDataFrame[['Drafted']].fillna(value="False")
    
    #Drops duplicate values
    mergedDataFrame = mergedDataFrame.drop_duplicates(subset=['Player'])
    
    #Drops columns not needed for the model
    mergedDataFrame = mergedDataFrame.drop(['School', 'College', 'Drafted (tm/rnd/yr)', 'AV', 'Pos_x'], axis=1)
    
    #Replacing draft values with numbers (1 for drafted, 0 for rejected)
    mergedDataFrame = mergedDataFrame.replace(to_replace =["True", "False"], 
                            value =[1, 0])
    
    #Here we want to find the unique position and sort it by either defense/offense
    offense = pd.read_csv("combined_offense.csv")
    defense = pd.read_csv("combined_defense.csv")
    
    offensive_positions = offense.Pos.unique()
    defensive_positions = defense.Pos.unique()

    #Offense and defense positions are discerned, 1 is offense and 0 is defense
    mergedDataFrame = mergedDataFrame.replace(to_replace = offensive_positions, 
                        value =1)
    
    mergedDataFrame = mergedDataFrame.replace(to_replace = defensive_positions, 
                        value =0)
        
    #Changes Height from a string to float and converts ft to in
    mergedDataFrame["Height"] = mergedDataFrame["Height"].apply(feet_to_inches)

    
    return mergedDataFrame

def webscrape_Salaries(start_year, end_year):
    site = "https://www.spotrac.com/nfl/salaries/breakdown/{}/"
    url = site.format(start_year)
    
    #Turns site info into list
    Scraped_Start = pd.read_html(url, header = [1])
    #Turns list into dataframe, remove other tables not needed
    Scraped_Start = Scraped_Start[0]
    
    #While loop to scrape data and place into a single CSV file
    while start_year <= end_year: 
        url = site.format(start_year+1)
        #Turns site info into list
        scraped_Next = pd.read_html(url, header = [1])
        #Turns list into dataframe, remove other tables not needed
        scraped_Next = scraped_Next[0]
        start_year+=1
        Scraped_Start = Scraped_Start.append(scraped_Next, ignore_index=True)
    
    #Save data onto computer
    Scraped_Start.to_csv('NFL Salaries from '+str(end_year)+'.csv', index=False)
    
    df_scraped = Scraped_Start
    return df_scraped

In [ ]:
#Combines datasets of metrics of draft player stats
combine_datasets(r'C:\Users\ThinkPad\Documents\Fall2021\QMB Data\NFL Combine\Defense_and_Offense', 'Defense_and_Offense' )

In [ ]:
#Scrapes draft website and places information into dataframe called "scraped"
scraped = webscrapingDraft(2000, 2017)

<ipython-input-2-d9840d0fe9bc>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scraped_segment["Drafted"] = "True"


In [ ]:
scraped

,Player,Pos,Drafted
0,Courtney Brown,DE,True
1,LaVar Arrington,LB,True
2,Chris Samuels,T,True
3,Peter Warrick,WR,True
4,Jamal Lewis,RB,True
...,...,...,...
4950,Rod Taylor,G,True
4951,Auden Tate,WR,True
4952,Korey Cunningham,T,True
4953,Austin Proehl,WR,True


In [ ]:
#Places combined dataframe (Combine stats) into a dataframe
combine_dataframe = pd.read_csv("combined_everything.csv")

In [ ]:
#Calls function "merge_and_clean"
a = merge_and_clean(scraped, combine_dataframe)

In [ ]:
a

,Player,Drafted,Rk,Year,Pos_y,Height,Wt,40YD,Vertical,BenchReps,Broad Jump,3Cone,Shuttle
0,Courtney Brown,1,98,2000,0,77.0,269,4.78,NaN,NaN,NaN,NaN,NaN
2,LaVar Arrington,1,110,2000,0,75.0,250,4.53,NaN,NaN,NaN,NaN,NaN
3,Chris Samuels,1,26,2000,1,77.0,325,5.08,NaN,NaN,NaN,NaN,NaN
4,Peter Warrick,1,9,2000,1,71.0,194,4.58,NaN,NaN,NaN,NaN,NaN
5,Jamal Lewis,1,50,2000,1,72.0,240,4.58,NaN,23.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5754,Javarius Leamon,0,159,2017,1,79.0,332,NaN,23.5,22.0,NaN,NaN,NaN
5755,Cameron Lee,0,160,2017,1,77.0,312,5.44,25.5,18.0,100.0,8.02,4.75
5756,Rushel Shell III,0,165,2017,1,70.0,227,4.74,32.5,21.0,NaN,NaN,NaN
5757,Nate Theaker,0,166,2017,1,77.0,315,5.41,27.0,23.0,105.0,7.97,4.89


In [ ]:
#Places the columns we want filled in into a list
zero_not_accepted = ['Height', 'Wt', '40YD', 'Vertical', 'BenchReps','Broad Jump', '3Cone', 'Shuttle']

In [ ]:
#Fills in all missing values with the mean of the associated column
for column in zero_not_accepted:
    a[column] = a[column].replace(0, np.NaN)
    mean = int(a[column].mean(skipna=True))
    a[column] = a[column].replace(np.NaN, mean)
    
a

,Player,Drafted,Rk,Year,Pos_y,Height,Wt,40YD,Vertical,BenchReps,Broad Jump,3Cone,Shuttle
0,Courtney Brown,1,98,2000,0,77.0,269,4.78,32.0,20.0,114.0,7.00,4.00
2,LaVar Arrington,1,110,2000,0,75.0,250,4.53,32.0,20.0,114.0,7.00,4.00
3,Chris Samuels,1,26,2000,1,77.0,325,5.08,32.0,20.0,114.0,7.00,4.00
4,Peter Warrick,1,9,2000,1,71.0,194,4.58,32.0,20.0,114.0,7.00,4.00
5,Jamal Lewis,1,50,2000,1,72.0,240,4.58,32.0,23.0,114.0,7.00,4.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5754,Javarius Leamon,0,159,2017,1,79.0,332,4.00,23.5,22.0,114.0,7.00,4.00
5755,Cameron Lee,0,160,2017,1,77.0,312,5.44,25.5,18.0,100.0,8.02,4.75
5756,Rushel Shell III,0,165,2017,1,70.0,227,4.74,32.5,21.0,114.0,7.00,4.00
5757,Nate Theaker,0,166,2017,1,77.0,315,5.41,27.0,23.0,105.0,7.97,4.89


In [ ]:
a = a.dropna()

In [ ]:
b = webscrape_Salaries(2000, 2017)
b = b[['Unnamed: 0','Average Salary']]
b.dropna()

,Unnamed: 0,Average Salary
0,Drew Bledsoe,"$6,000,000"
1,Michael Strahan,"$8,000,000"
2,Steve McNair,"$4,057,143"
3,Peyton Manning,"$7,719,000"
4,Edgerrin James,"$2,005,000"
...,...,...
21210,Keenan Robinson,"$835,000"
21211,Javarius Leamon,"$525,000"
21212,Rolan Milligan,"$525,000"
21214,Totals,"$2,731,843,495"


In [ ]:
b = b.rename(columns={'Unnamed: 0': 'Player'})
b

,Player,Average Salary
0,Drew Bledsoe,"$6,000,000"
1,Michael Strahan,"$8,000,000"
2,Steve McNair,"$4,057,143"
3,Peyton Manning,"$7,719,000"
4,Edgerrin James,"$2,005,000"
...,...,...
21211,Javarius Leamon,"$525,000"
21212,Rolan Milligan,"$525,000"
21213,Tyrunn Walker,NaN
21214,Totals,"$2,731,843,495"


In [ ]:
#Merge the two data frames (Salary and "Combine" info)
mergedDataFrame = pd.merge(b, a, on='Player', how='inner')
mergedDataFrame = mergedDataFrame.drop_duplicates(subset=['Player'])
df_TestSalary = mergedDataFrame

#Removes special characters in the salary column for modeling later
df_TestSalary['Average Salary'] = df_TestSalary['Average Salary'].str.replace('$', '')
df_TestSalary['Average Salary'] = df_TestSalary['Average Salary'].str.replace(',', '')
df_TestSalary['Average Salary'] = df_TestSalary['Average Salary'].astype(float)

#Rename column headers
df_TestSalary = df_TestSalary.rename(columns={'Average Salary': 'Average_Salary'})
df_TestSalary = df_TestSalary.dropna()
df_TestSalary

,Player,Average_Salary,Drafted,Rk,Year,Pos_y,Height,Wt,40YD,Vertical,BenchReps,Broad Jump,3Cone,Shuttle
0,Courtney Brown,3255929.0,1,98,2000,0,77.0,269,4.78,32.0,20.0,114.0,7.00,4.00
10,Dan Williams,4633333.0,1,11,2010,0,74.0,327,5.19,32.0,27.0,96.0,7.88,4.87
18,Peter Warrick,2435714.0,1,9,2000,1,71.0,194,4.58,32.0,20.0,114.0,7.00,4.00
24,Jamal Lewis,2373000.0,1,50,2000,1,72.0,240,4.58,32.0,23.0,114.0,7.00,4.00
34,LaVar Arrington,1893429.0,1,110,2000,0,75.0,250,4.53,32.0,20.0,114.0,7.00,4.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13448,Fadol Brown,480000.0,0,119,2017,0,76.0,273,4.94,34.0,19.0,115.0,7.24,4.44
13449,Hunter Sharp,525000.0,0,33,2016,1,71.0,198,4.58,32.5,12.0,116.0,7.12,4.19
13450,Dorian Johnson,525000.0,1,84,2017,1,77.0,300,5.27,30.0,21.0,114.0,8.39,5.09
13451,Alonzo Russell,540000.0,0,36,2016,1,76.0,206,4.54,29.5,12.0,112.0,7.18,4.33


In [ ]:
#Sectioning out the code to create the linear regression model

#Making linear regression model of predicting 40YD dash times

# https://www.youtube.com/watch?v=4HKqjENq9OU&t=590s
#Independent variables (Takes all rows and takes columns 9 and beyond)
X = df_TestSalary.iloc[:, 9:]
#Dependent variable: Average_Salary
y = df_TestSalary.iloc[:, 8]
#Testing the set, setting aside 20% of the values for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.3)


In [ ]:
#Multivariate linear regression model to determine if results of the combine will affect salary for NFL rookies
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [ ]:
#Model makes prediction using model created 
Y_pred = reg.predict(X_test)


In [ ]:
#Determining the model
print('Coefficients:', reg.coef_)
print('Intercept:', reg.intercept_)
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(y_test, Y_pred))
print('Coefficient of determination (R^2): %.2f'
      % r2_score(y_test, Y_pred))


Coefficients: [-0.01391133  0.00978459 -0.01115313  0.21599299  0.07238799]
Intercept: 4.416433443954269
Mean squared error (MSE): 0.03
Coefficient of determination (R^2): 0.62


In [ ]:
#A more organized result of the regression created
#Uses a different import than sklearn
#This is practically the same as the prior lines
reg = sm.OLS(df_TestSalary["40YD"], sm.add_constant(df_TestSalary[["Vertical","BenchReps","Broad Jump",
                                                                         "3Cone", "Shuttle", "Pos_y"]])).fit()

print(reg.summary())

                            OLS Regression Results                            
Dep. Variable:                   40YD   R-squared:                       0.579
Model:                            OLS   Adj. R-squared:                  0.578
Method:                 Least Squares   F-statistic:                     681.8
Date:                Sun, 12 Dec 2021   Prob (F-statistic):               0.00
Time:                        16:41:09   Log-Likelihood:                 633.93
No. Observations:                2986   AIC:                            -1254.
Df Residuals:                    2979   BIC:                            -1212.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.5393      0.148     30.576      0.0

In [ ]:
reg2 = sm.OLS(df_TestSalary["Drafted"], sm.add_constant(df_TestSalary[["Vertical","BenchReps","Broad Jump",
                                                                                 "40YD"]])).fit()

print(reg2.summary())

                            OLS Regression Results                            
Dep. Variable:                Drafted   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     16.30
Date:                Sun, 12 Dec 2021   Prob (F-statistic):           3.25e-13
Time:                        16:41:10   Log-Likelihood:                -1127.1
No. Observations:                2986   AIC:                             2264.
Df Residuals:                    2981   BIC:                             2294.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5739      0.247      2.327      0.0

In [ ]:
reg2 = sm.OLS(df_TestSalary["Average_Salary"], sm.add_constant(df_TestSalary[["Vertical","BenchReps","Broad Jump",
                                                                         "3Cone", "Shuttle", "Pos_y", "40YD"]])).fit()

print(reg2.summary())

                            OLS Regression Results                            
Dep. Variable:         Average_Salary   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     5.928
Date:                Sun, 12 Dec 2021   Prob (F-statistic):           7.26e-07
Time:                        16:41:11   Log-Likelihood:                -46672.
No. Observations:                2986   AIC:                         9.336e+04
Df Residuals:                    2978   BIC:                         9.341e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.266e+04   1.29e+06     -0.025      0.9

In [ ]:
reg2 = sm.OLS(df_TestSalary["Pos_y"], sm.add_constant(df_TestSalary[["Vertical","BenchReps","Broad Jump",
                                                                         "3Cone", "Shuttle", "40YD", "Height", "Wt"]])).fit()

print(reg2.summary())

                            OLS Regression Results                            
Dep. Variable:                  Pos_y   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.077
Method:                 Least Squares   F-statistic:                     31.98
Date:                Sun, 12 Dec 2021   Prob (F-statistic):           1.42e-48
Time:                        16:41:12   Log-Likelihood:                -2021.0
No. Observations:                2986   AIC:                             4060.
Df Residuals:                    2977   BIC:                             4114.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.8761      0.528     -3.556      0.0

Using KNN instead of a regression- KNN algorithm's purpose is to use a database in which the data points are separated into several classes to predict the classification of a new sample point. This works for categorical varibles such as the position of a player or whether or not a player was drafted. Based on feature similarity or how closely out-of-sample features resemble our training set determines how we classify a given data point

(https://medium.com/@adi.bronshtein/a-quick-introduction-to-k-nearest-neighbors-algorithm-62214cea29c7)

In [ ]:
#https://www.youtube.com/watch?v=4HKqjENq9OU&t=590s

#KNN model to determine whether or not the player was drafted

# https://www.youtube.com/watch?v=4HKqjENq9OU&t=590s
#Testing the values of the player attributes (Takes all rows and takes columns 5-13)
X = df_TestSalary.iloc[:, 5:14]
#Determing whether or not the player was drafted
y = df_TestSalary.iloc[:, 2]
#Testing the set, setting aside 20% of the values for later
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)

# Define the model: Init K-NN
#This will be the model that will be fitted to the data
classifier = KNeighborsClassifier(n_neighbors=10, p=1,metric='euclidean')

# Fit Model
classifier.fit(X_train, y_train)

# Predict the test set results
y_pred = classifier.predict(X_test)


In [ ]:
#This shows the accuracy of the model in predicting whether or not a player will be drafted
print(str(accuracy_score(y_test, y_pred)*100)+'%')

86.2876254180602%


In [ ]:
#Indication of false positives in the model
print(f1_score(y_test, y_pred))

0.9259927797833936


In [ ]:
#KNN model to determine the position of the player

# https://www.youtube.com/watch?v=4HKqjENq9OU&t=590s
#Testing the values of the player attributes (Takes all rows and takes columns 10-17)
A = df_TestSalary.iloc[:, 6:14]
#Determing whether or not the player was drafted
B = df_TestSalary.iloc[:, 5]
#Testing the set, setting aside 20% of the values for later
X_train, X_test, y_train, y_test = train_test_split(A, B, random_state=0, test_size=0.2)

# Define the model: Init K-NN
#This will be the model that will be fitted to the data
classifier1 = KNeighborsClassifier(n_neighbors=10, p=1,metric='euclidean')

# Fit Model
classifier1.fit(X_train, y_train)

# Predict the test set results
y_pred = classifier1.predict(X_test)


In [ ]:
#This shows the accuracy of the model in predicting whether or not a player will be drafted
print(str(accuracy_score(y_test, y_pred)*100)+'%')

69.7324414715719%


In [ ]:
#Indication of false positives in the model (Higher the better)
print(f1_score(y_test, y_pred))

0.5895691609977325
